#Tokenizer

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
veri=[
    "Bu bir örnek cümle.",
    "Başka bir cümle de var.",
    "Farklo bir örnek metin."
]

In [3]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(veri)

In [8]:
sayisal_karsilik=tokenizer.texts_to_sequences(veri)
print("kelime indexleri:",tokenizer.word_index)

kelime indexleri: {'bir': 1, 'örnek': 2, 'cümle': 3, 'bu': 4, 'başka': 5, 'de': 6, 'var': 7, 'farklo': 8, 'metin': 9}


In [9]:
for sayi in sayisal_karsilik:
  print(sayi)

[4, 1, 2, 3]
[5, 1, 3, 6, 7]
[8, 1, 2, 9]


In [14]:
sabit_uzunluk=pad_sequences(sayisal_karsilik,maxlen=5 )#,padding="post")
print(sabit_uzunluk)

[[0 4 1 2 3]
 [5 1 3 6 7]
 [0 8 1 2 9]]


#Embedding

In [15]:
from keras.layers import Embedding
import numpy as np

In [16]:
giris_boyutu=len(tokenizer.word_index)+1
boyut=300
embdedding_layer=Embedding(input_dim=giris_boyutu,output_dim=boyut,input_length=5)
kimlikli_veri=embdedding_layer(np.array(sabit_uzunluk))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
print("Embedding katman sonucu:")
print(kimlikli_veri)

Embedding katman sonucu:
tf.Tensor(
[[[ 0.04412032  0.04519786 -0.03040048 ...  0.02029041 -0.02904425
    0.01586807]
  [-0.03727756 -0.00378201 -0.03370059 ... -0.00318452  0.03292427
   -0.0497348 ]
  [ 0.03704808 -0.04379463  0.03045635 ... -0.00939263  0.02670044
   -0.01724154]
  [-0.03716528  0.04062948  0.01095844 ... -0.00235326 -0.04383833
   -0.01361959]
  [ 0.00501233 -0.0170051   0.04901711 ...  0.02548664  0.03445233
   -0.01056542]]

 [[-0.00695525 -0.04825382 -0.00727723 ...  0.01866745 -0.03600943
   -0.02515315]
  [ 0.03704808 -0.04379463  0.03045635 ... -0.00939263  0.02670044
   -0.01724154]
  [ 0.00501233 -0.0170051   0.04901711 ...  0.02548664  0.03445233
   -0.01056542]
  [-0.00556792  0.04803808  0.00598885 ... -0.0341977  -0.02626592
   -0.02532868]
  [ 0.02726218  0.02896312 -0.03121222 ... -0.02385929 -0.01475676
    0.04915963]]

 [[ 0.04412032  0.04519786 -0.03040048 ...  0.02029041 -0.02904425
    0.01586807]
  [ 0.01289562 -0.00934569  0.01430947 ...  0.0

In [49]:
from google.colab import drive
!pip install joblib
!pip install colorama
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from keras.layers import SimpleRNN,Dense
from keras.utils import to_categorical
import joblib
from colorama import Fore,Back,Style


drive.mount('/content/drive')
model_yolu='/content/drive/MyDrive/Colab Notebooks/rnn.h5'
veri_yol="/content/drive/MyDrive/Colab Notebooks/Phishing_Email.csv"
print(veri_yol)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Phishing_Email.csv


In [29]:
mail_verisi=pd.read_csv(veri_yol)
mail_verisi=mail_verisi.where((pd.notnull(mail_verisi)),"")
mail_verisi.loc[mail_verisi["Email Type"]=="Safe Email","Email Type"]=0
mail_verisi.loc[mail_verisi["Email Type"]=="Phishing Email","Email Type"]=1


In [30]:
X=mail_verisi["Email Text"]
y=mail_verisi["Email Type"]

In [ ]:
#Tokenizer

In [31]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X)
indeksli_veri=tokenizer.texts_to_sequences(X)

word_index=tokenizer.word_index
kelime_uzunlugu=100
sabit_uzunluk=pad_sequences(indeksli_veri,maxlen=kelime_uzunlugu)


In [32]:
y=to_categorical(y)

In [28]:
print("Kelime indexleri:")
print(word_index)

Output hidden; open in https://colab.research.google.com to view.

#RNN

In [35]:
X_train,X_test,y_train,y_test=train_test_split(sabit_uzunluk,y,test_size=0.2,random_state=1)


In [36]:
model=Sequential()
model.add(Embedding(len(word_index)+1,output_dim=128,input_length=kelime_uzunlugu))
model.add(SimpleRNN(64,activation="relu"))
model.add(Dense(2,activation="softmax"))
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [37]:
model.fit(X_train,y_train,epochs=5,batch_size=32,validation_data=(X_test,y_test))

Epoch 1/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 161s 338ms/step - accuracy: 0.8201 - loss: 0.3466 - val_accuracy: 0.9641 - val_loss: 0.0976
Epoch 2/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 159s 340ms/step - accuracy: 0.9833 - loss: 0.0467 - val_accuracy: 0.9633 - val_loss: 0.0934
Epoch 3/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 202s 340ms/step - accuracy: 0.9681 - loss: 0.1133 - val_accuracy: 0.9558 - val_loss: 0.1354
Epoch 4/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 203s 341ms/step - accuracy: 0.9874 - loss: 0.0282 - val_accuracy: 0.9576 - val_loss: 0.1180
Epoch 5/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 202s 341ms/step - accuracy: 0.9874 - loss: 0.0320 - val_accuracy: 0.9550 - val_loss: 0.1537


In [38]:
loss,accuracy=model.evaluate(X_test,y_test)
print("Kayıp:",loss)
print("Başarı:",accuracy)

117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9505 - loss: 0.1597
Kayıp: 0.15369082987308502
Başarı: 0.9549598097801208


In [50]:
model.save(model_yolu)

In [51]:
while True:
  giris= input("maili girin")
  sayısal_kelime=tokenizer.texts_to_sequences([giris])
  sabit_uzunluk = pad_sequences(sayısal_kelime, maxlen=kelime_uzunlugu)
  tahmin=model.predict(sabit_uzunluk)
  if np.argmax(tahmin)==1:
    print(Fore.RED,"Bu bir phishing mailidir.")
  else:
    print(Fore.GREEN,"Bu bir güvenli emaildir")
  print(Fore.WHITE,"Doğruluk puanı:",tahmin[0][np.argmax(tahmin)])

maili girin PROMOTE YOUR PRODUCT OR SERVICE TO MILLIONS TODAY! Â  E-MAIL MARKETING - Bulk e-mail will make you money so fast, your head will spin!- Our customers tell us that they would no longer be in business without it.- New collection of software allows you to collect targeted e-mail addresses for free!Â Â Â  See this product's web page CLICK HERE Â  1 MILLION AMERICAN BUSINESS LEADS ON CD- If you do telemarketing, mailing, or faxing this list will be a gold mine! - Contains company name, address, phone, fax, SIC, # of employees & revenue - List allows for UNLIMITED DOWNLOADS! Â Â Â  See this product's web page  CLICK HERE Â  FAX MARKETING SYSTEM - Fax broadcasting is the hot new way to market your product or service!- People are 4 times more likely to read faxes than direct mail.- Software turns your computer into a fax blaster with 4 million  leads on disk! Â Â Â  See this product's web page  CLICK HERE Â  Visit our web site or call 618-288-6661 for more information. Â  Â  Â  Â t

KeyboardInterrupt: Interrupted by user